In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

<b><font size="6" color="#009e84"> ch9. 10장 Retrieval의 효율개선을 위한 전처리(table) </font></b>

# 벡터DB : Chroma vs Pinecone

- Chroma : 인메모리 vector DB, 로컬 vector DB
- Pinecone : 클라우드 vector DB
    - https://www.pinecone.io 에서 api key 생성 → .env에 추가(PINECONE_API_KEY 등록)

# 0. 패키지 설치

In [3]:
%pip install -q pinecone langchain-pinecone --no-warn-script-location

Note: you may need to restart the kernel to use updated packages.


# 1. knowledge Base 구성을 위한 데이터 생성

In [2]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = Docx2txtLoader('data/소득세법_with table.docx')
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500,
                                               chunk_overlap=200)
document_list = loader.load_and_split(text_splitter=text_splitter)
len(document_list)

194

In [3]:
# embedding : upstage의 solar-embedding-1-large-passage

from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings
load_dotenv()

embedding = UpstageEmbeddings(model='solar-embedding-1-large-passage')

In [4]:
len(embedding.embed_query('소득세법'))

4096

In [5]:
document_list[46].page_content

'제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n종합소득 과세표준\n\n세율\n\n1,400만원 이하\n\n과세표준의 6퍼센트\n\n1,400만원 초과 5,000만원 이하\n\n84만원 + (1,400만원을 초과하는 금액의 15퍼센트)\n\n5,000만원 초과 8,800만원 이하\n\n624만원 + (5,000만원을 초과하는 금액의 24퍼센트)\n\n8,800만원 초과 1억5천만원 이하\n\n1,536만원 + (8,800만원을 초과하는 금액의 35퍼센트)\n\n1억5천만원 초과 3억원 이하\n\n3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)\n\n3억원 초과 5억원 이하\n\n9,406만원 + (3억원을 초과하는 금액의 40퍼센트)\n\n5억원 초과 10억원 이하\n\n1억7,406만원 + (5억원을 초과하는 금액의 42퍼센트)\n\n10억원 초과\n\n3억8,406만원 + (10억원을 초과하는 금액의 45퍼센트)\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한다)으로 한다.<개정 2013. 1. 1., 2014. 12. 23.>\n\n1. 해당 과세기간의 퇴직소득과세표준에 제1항의 세율을 적용하여 계산한 금액\n\n2. 제1호의 금액을 12로 나눈 금액에 근속연수를 곱한 금액\n\n3. 삭제<2014. 12. 23.>\n\n[전문개정 2009. 12. 31.]\n\n\n\n제2관 세액공제 <개정 2009. 12. 31.>\n\n\n\n제56조(배당세액공제) ① 거주자의 종합소득금액에 제17조제3항 각 호 외의 부분 단서가 적용되는 배당소득금액이 합산되어 있는 경우에는 같은 항 각 호 외의 부분 

In [6]:
%%time

# pinecone vector database 저장
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
import os

pc = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))

# 업로드시 시간이 오래걸려서 경고가 뜨는데 안보이려면 아나콘다 프롬프트 llm 환경에서 conda install -c conda-forge ipywidgets

# 데이터를 처음 업로드할 때
index_name = 'tax-index-table'
# database = PineconeVectorStore.from_documents(documents=document_list,
#                                               embedding=embedding,
#                                               index_name=index_name)

# 업로드한 벡터DB를 가져올 때
database = PineconeVectorStore(embedding=embedding, # 질문을 임베딩하여 유사도 검색
                               index_name=index_name)

CPU times: total: 6.53 s
Wall time: 1min 4s


# 2. 답변 생성 전 retrieval 확인

In [7]:
query = '연봉이 5천만원인 직장인의 소득세는 얼마인가요?'
retrieved_docs = database.similarity_search(query, k=3) # 기본 k 값 : 4

In [17]:
retrieved_docs[1].page_content

'제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n종합소득 과세표준\n\n세율\n\n1,400만원 이하\n\n과세표준의 6퍼센트\n\n1,400만원 초과 5,000만원 이하\n\n84만원 + (1,400만원을 초과하는 금액의 15퍼센트)\n\n5,000만원 초과 8,800만원 이하\n\n624만원 + (5,000만원을 초과하는 금액의 24퍼센트)\n\n8,800만원 초과 1억5천만원 이하\n\n1,536만원 + (8,800만원을 초과하는 금액의 35퍼센트)\n\n1억5천만원 초과 3억원 이하\n\n3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)\n\n3억원 초과 5억원 이하\n\n9,406만원 + (3억원을 초과하는 금액의 40퍼센트)\n\n5억원 초과 10억원 이하\n\n1억7,406만원 + (5억원을 초과하는 금액의 42퍼센트)\n\n10억원 초과\n\n3억8,406만원 + (10억원을 초과하는 금액의 45퍼센트)\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한다)으로 한다.<개정 2013. 1. 1., 2014. 12. 23.>\n\n1. 해당 과세기간의 퇴직소득과세표준에 제1항의 세율을 적용하여 계산한 금액\n\n2. 제1호의 금액을 12로 나눈 금액에 근속연수를 곱한 금액\n\n3. 삭제<2014. 12. 23.>\n\n[전문개정 2009. 12. 31.]\n\n\n\n제2관 세액공제 <개정 2009. 12. 31.>\n\n\n\n제56조(배당세액공제) ① 거주자의 종합소득금액에 제17조제3항 각 호 외의 부분 단서가 적용되는 배당소득금액이 합산되어 있는 경우에는 같은 항 각 호 외의 부분 

In [18]:
# retrieved_docs[0].page_content

retrieved_doc = '\n\n---\n\n'.join([doc.page_content for doc in retrieved_docs])

In [19]:
# 두칸 위에있는거랑 완벽하게 같음

retriever = database.as_retriever(search_kwargs={'k':3})
retrieved_docs = retriever.invoke(query)

In [22]:
retrieved_docs[1].page_content

'제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n종합소득 과세표준\n\n세율\n\n1,400만원 이하\n\n과세표준의 6퍼센트\n\n1,400만원 초과 5,000만원 이하\n\n84만원 + (1,400만원을 초과하는 금액의 15퍼센트)\n\n5,000만원 초과 8,800만원 이하\n\n624만원 + (5,000만원을 초과하는 금액의 24퍼센트)\n\n8,800만원 초과 1억5천만원 이하\n\n1,536만원 + (8,800만원을 초과하는 금액의 35퍼센트)\n\n1억5천만원 초과 3억원 이하\n\n3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)\n\n3억원 초과 5억원 이하\n\n9,406만원 + (3억원을 초과하는 금액의 40퍼센트)\n\n5억원 초과 10억원 이하\n\n1억7,406만원 + (5억원을 초과하는 금액의 42퍼센트)\n\n10억원 초과\n\n3억8,406만원 + (10억원을 초과하는 금액의 45퍼센트)\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한다)으로 한다.<개정 2013. 1. 1., 2014. 12. 23.>\n\n1. 해당 과세기간의 퇴직소득과세표준에 제1항의 세율을 적용하여 계산한 금액\n\n2. 제1호의 금액을 12로 나눈 금액에 근속연수를 곱한 금액\n\n3. 삭제<2014. 12. 23.>\n\n[전문개정 2009. 12. 31.]\n\n\n\n제2관 세액공제 <개정 2009. 12. 31.>\n\n\n\n제56조(배당세액공제) ① 거주자의 종합소득금액에 제17조제3항 각 호 외의 부분 단서가 적용되는 배당소득금액이 합산되어 있는 경우에는 같은 항 각 호 외의 부분 

# 3. 답변 생성

In [23]:
from openai import OpenAI

client = OpenAI()
response = client.chat.completions.create(model='gapt-4o-mini',
                                          messages=[{'role':'system', 'content':'당신은 최고의 한국 소득세법 전문가입니다'},
                                                    {'role':'user', 'content':f'''-[context]를 참고해서 사용자의 질문에 10줄 이내로 답변해주세요
                                                     -[context] : {retrieved_doc}
                                                     -질문 : {query}'''}],
                                          temperature=0.2)

In [28]:
print(response.choices[0].message.content)

연봉이 5천만원인 직장인의 소득세를 계산하기 위해, 먼저 종합소득 과세표준에 따른 세율을 적용합니다.

1. 5천만원의 경우, 과세표준은 다음과 같이 계산됩니다:
   - 1,400만원 이하: 1,400만원 × 6% = 84만원
   - 1,400만원 초과 5,000만원 이하: 84만원 + (5,000만원 - 1,400만원) × 15% = 84만원 + 525만원 = 609만원

2. 근로소득세액공제를 적용합니다:
   - 총급여액이 3천 300만원 초과 5천만원 이하이므로, 공제액은 74만원 - [(5,000만원 - 3,300만원) × 8/1000] = 74만원 - 13.6만원 = 60.4만원 (단, 66만원보다 적지 않으므로 66만원 적용).

3. 최종 소득세는 609만원 - 66만원 = 543만원입니다.

따라서, 연봉이 5천만원인 직장인의 소득세는 약 543만원입니다.


In [29]:
from openai import OpenAI
import os

client = OpenAI(api_key=os.getenv('UPSTAGE_API_KEY'),
                base_url="https://api.upstage.ai/v1")
response = client.chat.completions.create(model='solar-pro2',
                                          messages=[{'role':'system', 'content':'당신은 최고의 한국 소득세법 전문가입니다'},
                                                    {'role':'user', 'content':f'''-[context]를 참고해서 사용자의 질문에 10줄 이내로 답변해주세요
                                                     -[context] : {retrieved_doc}
                                                     -질문 : {query}'''}],
                                          temperature=0.2)

In [31]:
print(response.choices[0].message.content)

연봉 5천만원의 소득세 계산은 다음과 같습니다:  

1. **종합소득과세표준**: 5,000만원 - 근로소득 공제(1,275만원) = **3,725만원**  
   (※ 근로소득 공제: 5,000만원 × 25% - 330만원)  

2. **종합소득산출세액**:  
   - 1,400만원 × 6% = 84만원  
   - (3,725만원 - 1,400만원) × 15% = 363만원  
   - **총 산출세액**: 84 + 363 = **447만원**  

3. **근로소득세액공제**:  
   - 74만원 - [(5,000만원 - 3,300만원) × 8/1,000] = **70.88만원** (66만원 미만 시 66만원 적용)  

4. **최종 세액**: 447만원 - 70.88만원 = **376.12만원**  

※ 추가 공제(자녀, 의료비 등) 적용 시 세액 감소 가능합니다.  

(※ 계산은 기본 공제만 반영, 실제 세액은 공제 항목에 따라 달라질 수 있음)


In [9]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4.1-nano')

In [32]:
# upstage에서 받은 20$로 llm을 사용하고 싶다면

from langchain_upstage import ChatUpstage

llm = ChatUpstage(model = 'solar-pro2',
                  reasoning_effort='high') # 느리지만 더 깊게 추론함 (low, medium)

In [33]:
prompt = f'''[identity]
-당신은 최고의 한국 소득세법 전문가입니다
-[context]를 참고해서 사용자의 질문에 답변해주세요
-[context]는 다음과 같습니다
{retrieved_doc}
-질문 : {query}'''

In [34]:
ai_message = llm.invoke(prompt)

In [36]:
print(ai_message.content)

연봉 5천만원인 직장인의 소득세를 계산하기 위해 다음과 같은 단계를 따릅니다. 단, 질문에서 공제 항목(예: 보험료, 의료비, 자녀세액공제 등)에 대한 정보가 제공되지 않았으므로 **근로소득세액공제만 적용**하여 계산합니다.

---

### 1. **종합소득산출세액 계산**  
총급여액 5천만원을 기준으로 종합소득산출세액을 계산합니다.  
한국 소득세율(법 제55조)에 따라 다음과 같이 적용됩니다:

| 과세표준 구간               | 세율 계산식                          | 산출세액 |
|----------------------------|--------------------------------------|----------|
| 1,400만원 이하             | 1,400만원 × 6% = 84만원              | 84만원   |
| 1,400만원 초과 ~ 5,000만원 | 84만원 + (5,000만원 - 1,400만원) × 15% | 624만원   |
| **5,000만원 초과 ~ 8,800만원** | **624만원 + (5,000만원 초과분) × 24%** | **624만원** (5천만원 초과분 없음) |

- **5천만원 초과분**: 5,000만원 - 5,000만원 = 0원  
- **종합소득산출세액**: **624만원**

---

### 2. **근로소득세액공제 적용**  
총급여액 5천만원은 **3,300만원 초과 ~ 7,000만원 이하** 구간에 해당하므로, 공제액은 다음과 같이 계산됩니다(법 제59조 제1항, 제2항):

1. **기본공제액**: 74만원  
2. **초과분 계산**:  
   - (5,000만원 - 3,300만원) × 8/1,000 = 1,700만원 × 0.008 = **13.6만원**  
   - **공제액**: 74만원 - 13.6만원 = **60.4만원**  
3. **최소공제액 적용**: 60.4만원 < 66만원이므로 **66만원**으로 조정.

- **근로소득세액

# 4. langchain으로 답변 생성

In [39]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_upstage import ChatUpstage
from dotenv import load_dotenv
load_dotenv()

# 1. LLM과 임베딩 초기화
# llm = ChatOpenAI(model='gpt-4.1-mini')
llm = ChatUpstage(model='solar-pro2')
embedding = UpstageEmbeddings(model='solar-embedding-1-large-passage')

# 2. vertor store load
index_name = 'tax-index-table'
vectorstore = PineconeVectorStore(embedding=embedding, 
                               index_name=index_name)

# 3. Retriever 생성
retriever = vectorstore.as_retriever(search_type='similarity',
                                     search_kwargs={'k':4})

# 4. 프롬프트 템플릿
template = f'당신은 최고의 한국 소득세 전문가입니다. 다음 문맥을 참고하여 질문에 답하세요. 답을 모르면 모른다고 답하세요. 최대 3문장으로 간결하게 답변하세요. 질문 : {{query}} / 문맥 : {{context}} / 답변 :'
prompt = ChatPromptTemplate.from_template(template)

# 5. 검색된 document를 텍스트로 변환하는 함수
def format_documents(documents):
    return '\n\n---\n\n'.join([doc.page_content for doc in documents])

In [40]:
# 6. RAG 체인 구성 (LCEL 방식)
from langchain_core.runnables import RunnablePassthrough # {'query':'~'} → '~'
rag_chain = ({'context':retriever | format_documents, 'query':RunnablePassthrough()} # 질문 그대로 전달
             | prompt # prompt에 context와 query 변수 주입
             | llm    # llm에 prompt invoke
             | StrOutputParser()) 

# 7. 실행
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
rag_chain.invoke(query)

'주어진 문맥은 근로소득 세액공제(제59조)를 설명하고 있습니다. 연봉 5천만원인 직장인의 경우, 총급여액이 3,300만원 초과 7,000만원 이하에 해당하므로 다음과 같이 계산됩니다:  \n**74만원 - [(5,000만원 - 3,300만원) × 8/1000] = 74만원 - 1,440만원 × 0.008 = 74만원 - 11.52만원 = 62.48만원**  \n하지만 최소 공제액이 66만원이므로, **근로소득세액공제액은 66만원**입니다. 단, 이는 세액공제액이며, 실제 소득세는 종합소득과세표준(제55조)에 따라 별도로 계산됩니다.  \n\n(답변은 근로소득 세액공제액 기준이며, 종합소득세는 추가 정보 필요)'

# 5. 성능

In [1]:
# 업로드한 벡터 db를 가져오기

from dotenv import load_dotenv
from pinecone import Pinecone
from langchain_upstage import UpstageEmbeddings
from langchain_pinecone import PineconeVectorStore

load_dotenv()
pc = Pinecone()
embedding = UpstageEmbeddings(model='solar-embedding-1-large-passage')
# Pinecone 객체가 없으면 이것만 한다고 못가져옴.
vectorstore = PineconeVectorStore(embedding=embedding,
                                  index_name='tax-index-upstage') 

In [2]:
retriever = vectorstore.as_retriever(search_type='similarity', # 기본. 유사한것을 4개 가져와라고 하는것. 그런데 만약 내가 DB에 똑같은 정보를 2번 넣으면 사실상 2개만 가져오는것. mmr은 같은것제외
                                     search_kwargs={'k':4})    # 기본
retrieved_docs = retriever.invoke('연봉 5천만원인 직장인의 소득세는 얼마인가요?')

In [8]:
for doc in retrieved_docs:
    print(doc.page_content)
    print('='*100)

2. 2명인 경우: 연 55만원

3. 3명 이상인 경우: 연 55만원과 2명을 초과하는 1명당 연 40만원을 합한 금액

② 삭제<2017. 12. 19.>

③ 해당 과세기간에 출산하거나 입양 신고한 공제대상자녀가 있는 경우 다음 각 호의 구분에 따른 금액을 종합소득산출세액에서 공제한다.<신설 2015. 5. 13., 2016. 12. 20.>

1. 출산하거나 입양 신고한 공제대상자녀가 첫째인 경우: 연 30만원

2. 출산하거나 입양 신고한 공제대상자녀가 둘째인 경우: 연 50만원

3. 출산하거나 입양 신고한 공제대상자녀가 셋째 이상인 경우: 연 70만원

④ 제1항 및 제3항에 따른 공제를 “자녀세액공제”라 한다.<신설 2015. 5. 13., 2017. 12. 19.>

[본조신설 2014. 1. 1.]

[종전 제59조의2는 제59조의5로 이동 <2014. 1. 1.>]



제59조의3(연금계좌세액공제) ① 종합소득이 있는 거주자가 연금계좌에 납입한 금액 중 다음 각 호에 해당하는 금액을 제외한 금액(이하 “연금계좌 납입액”이라 한다)의 100분의 12[해당 과세기간에 종합소득과세표준을 계산할 때 합산하는 종합소득금액이 4천 500만원 이하(근로소득만 있는 경우에는 총급여액 5천 500만원 이하)인 거주자에 대해서는 100분의 15]에 해당하는 금액을 해당 과세기간의 종합소득산출세액에서 공제한다. 다만, 연금계좌 중 연금저축계좌에 납입한 금액이 연 600만원을 초과하는 경우에는 그 초과하는 금액은 없는 것으로 하고, 연금저축계좌에 납입한 금액 중 600만원 이내의 금액과 퇴직연금계좌에 납입한 금액을 합한 금액이 연 900만원을 초과하는 경우에는 그 초과하는 금액은 없는 것으로 한다. <개정 2014. 12. 23., 2015. 5. 13., 2016. 12. 20., 2022. 12. 31.>

1. 제146조제2항에 따라 소득세가 원천징수되지 아니한 퇴직소득 등 과세가 이연된 소득

2. 연금계좌에서 다른 연금계좌로 계약을 이전함으로써 납입되

In [10]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
retrieved_docs = vectorstore.similarity_search(query, k=3)
for doc in retrieved_docs:
    print(doc.page_content)
    print('='*100)

2. 2명인 경우: 연 55만원

3. 3명 이상인 경우: 연 55만원과 2명을 초과하는 1명당 연 40만원을 합한 금액

② 삭제<2017. 12. 19.>

③ 해당 과세기간에 출산하거나 입양 신고한 공제대상자녀가 있는 경우 다음 각 호의 구분에 따른 금액을 종합소득산출세액에서 공제한다.<신설 2015. 5. 13., 2016. 12. 20.>

1. 출산하거나 입양 신고한 공제대상자녀가 첫째인 경우: 연 30만원

2. 출산하거나 입양 신고한 공제대상자녀가 둘째인 경우: 연 50만원

3. 출산하거나 입양 신고한 공제대상자녀가 셋째 이상인 경우: 연 70만원

④ 제1항 및 제3항에 따른 공제를 “자녀세액공제”라 한다.<신설 2015. 5. 13., 2017. 12. 19.>

[본조신설 2014. 1. 1.]

[종전 제59조의2는 제59조의5로 이동 <2014. 1. 1.>]



제59조의3(연금계좌세액공제) ① 종합소득이 있는 거주자가 연금계좌에 납입한 금액 중 다음 각 호에 해당하는 금액을 제외한 금액(이하 “연금계좌 납입액”이라 한다)의 100분의 12[해당 과세기간에 종합소득과세표준을 계산할 때 합산하는 종합소득금액이 4천 500만원 이하(근로소득만 있는 경우에는 총급여액 5천 500만원 이하)인 거주자에 대해서는 100분의 15]에 해당하는 금액을 해당 과세기간의 종합소득산출세액에서 공제한다. 다만, 연금계좌 중 연금저축계좌에 납입한 금액이 연 600만원을 초과하는 경우에는 그 초과하는 금액은 없는 것으로 하고, 연금저축계좌에 납입한 금액 중 600만원 이내의 금액과 퇴직연금계좌에 납입한 금액을 합한 금액이 연 900만원을 초과하는 경우에는 그 초과하는 금액은 없는 것으로 한다. <개정 2014. 12. 23., 2015. 5. 13., 2016. 12. 20., 2022. 12. 31.>

1. 제146조제2항에 따라 소득세가 원천징수되지 아니한 퇴직소득 등 과세가 이연된 소득

2. 연금계좌에서 다른 연금계좌로 계약을 이전함으로써 납입되

In [ ]:
# query에 가장 유사도가 높은 55조는 이미지로 되어있어 chunk에 미포함 → 이미지를 table로 변경(ch9.10예제)